In [ ]:
# !git clone https://github.com/JrDKHub/Immostats.git
# %pwd

In [ ]:
# %ls -l
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import seaborn as sn


file_path1 = 'data_2025.txt'
df1 = pd.read_csv(
    file_path1, 
    sep='|', 
    decimal=',',
    dtype={
        'Code postal': str, 
        'Code commune': str, 
        'Code departement': str
    },
    low_memory=False
)

# données de 2024
file_path2 = 'data_house.txt'
# sep='|' : Indispensable car votre fichier n'utilise pas des virgules comme séparateur
# decimal=',' : Indispensable pour que "346,50" soit compris comme un nombre et pas du texte
# dtype : Indispensable pour garder '01' (Ain) et pas '1'
df2 = pd.read_csv(
    file_path2, 
    sep='|', 
    decimal=',',
    dtype={
        'Code postal': str, 
        'Code commune': str, 
        'Code departement': str
    }
)
# on melange les données de 2025 et 2024
df = pd.concat([df1, df2], ignore_index=True)
# Vérification
print(f"Dimensions : {df.shape}")
df.head(2)

In [ ]:
DTYPE_MAPPING = {
    'Code postal': str,
    'Code commune': str,
    'Code departement': str,
    'Code voie': str
}

COLUMN_MAPPING = {
    # --- Identifiants & Administratif ---
    'Identifiant de document': 'id_mutation',
    'Reference document': 'reference_document',
    '1 Articles CGI': 'article_cgi_1',
    '2 Articles CGI': 'article_cgi_2',
    '3 Articles CGI': 'article_cgi_3',
    '4 Articles CGI': 'article_cgi_4',
    '5 Articles CGI': 'article_cgi_5',
    'No disposition': 'numero_disposition',
    'Date mutation': 'date_mutation',
    'Nature mutation': 'nature_mutation',
    'Valeur fonciere': 'valeur_fonciere',
    
    # --- Adresse & Géographie ---
    'No voie': 'numero_voie',
    'B/T/Q': 'btq', # Bis, Ter, Quater
    'Type de voie': 'type_voie',
    'Code voie': 'code_voie',
    'Voie': 'nom_voie',
    'Code postal': 'code_postal',
    'Commune': 'commune',
    'Code departement': 'code_departement',
    'Code commune': 'code_commune',
    
    # --- Cadastre ---
    'Prefixe de section': 'prefixe_section',
    'Section': 'section',
    'No plan': 'numero_plan',
    'No Volume': 'numero_volume',
    
    # --- Lots de copropriété (1 à 5) ---
    '1er lot': 'lot1_numero',
    'Surface Carrez du 1er lot': 'lot1_surface_carrez',
    '2eme lot': 'lot2_numero',
    'Surface Carrez du 2eme lot': 'lot2_surface_carrez',
    '3eme lot': 'lot3_numero',
    'Surface Carrez du 3eme lot': 'lot3_surface_carrez',
    '4eme lot': 'lot4_numero',
    'Surface Carrez du 4eme lot': 'lot4_surface_carrez',
    '5eme lot': 'lot5_numero',
    'Surface Carrez du 5eme lot': 'lot5_surface_carrez',
    'Nombre de lots': 'nombre_lots',
    
    # --- Caractéristiques du Bien ---
    'Code type local': 'code_type_local',
    'Type local': 'type_local',
    'Identifiant local': 'id_local',
    'Surface reelle bati': 'surface_reelle_bati',
    'Nombre pieces principales': 'nombre_pieces_principales',
    'Nature culture': 'nature_culture',
    'Nature culture speciale': 'nature_culture_speciale',
    'Surface terrain': 'surface_terrain'
}

# on converti les données de date au format d-m-y
if 'Date mutation' in df.columns:
        df['Date mutation'] = pd.to_datetime(df['Date mutation'], format='%d/%m/%Y')

df.rename(columns=COLUMN_MAPPING, inplace=True)
df.columns

In [ ]:
cols_to_drop = [
    "id_mutation", "reference_document",
    "article_cgi_1", "article_cgi_2", "article_cgi_3", "article_cgi_4", "article_cgi_5",
    "numero_disposition",
    "numero_voie", "btq", "type_voie", "code_voie", "nom_voie",
    "prefixe_section", "section", "numero_plan", "numero_volume",
    "lot1_numero", "lot2_numero", "lot3_numero", "lot4_numero", "lot5_numero",
    "id_local",
    "nature_culture", "nature_culture_speciale",
    "nombre_lots",
]

# On supprime les colonnes
df = df.drop(columns=cols_to_drop, errors="ignore")

cols_to_drop = [
    "nature_mutation",
    "type_local",
    "lot1_surface_carrez",
    "lot2_surface_carrez",
    "lot3_surface_carrez",
    "lot4_surface_carrez",
    "lot5_surface_carrez",
]

df = df.drop(columns=cols_to_drop, errors="ignore")
df.describe()


In [ ]:
numeric_df = df.select_dtypes(include=np.number)
param_stat = numeric_df.agg(['median', 'kurt', 'var', 'skew'])
param_stat

In [ ]:
dfCorr = df.corr(numeric_only= True)
dfCorr
sn.heatmap(data = dfCorr, annot = True)

In [ ]:
# supression des lignes en redondances
duplicates = df.duplicated().sum()
if duplicates > 0:
    df.drop_duplicates(inplace=True)
print(f"Number of duplicates removed: {duplicates}")

In [ ]:
# Recherche des valeurs manquantes
missing_values = df.isnull().sum()
print("Missing values:\n", missing_values)

In [ ]:
# fonction pour mapper mois -> saison (France, météo)
def get_season(m):
    if m in [12, 1, 2]:
        return "hiver"
    elif m in [3, 4, 5]:
        return "printemps"
    elif m in [6, 7, 8]:
        return "ete"
    elif m in [9, 10, 11]:
        return "automne"
    else:
        return None

# Convertion de la date de mutaion en trois sous colonnes (annee, mois, jour)
# Vérifier que la colonne existe
if 'date_mutation' in df.columns:

    # Conversion en datetime
    df['date_mutation'] = pd.to_datetime(df['date_mutation'], errors='coerce')

    # Vérifier si certaines dates n'ont pas été converties
    if df['date_mutation'].isnull().sum() > 0:
        print("Certaines valeurs n'ont pas pu être converties en date.")

    # Extraction des composantes temporelles
    df['annee_mutation'] = df['date_mutation'].dt.year
    df['mois_mutation'] = df['date_mutation'].dt.month
    df['jour_mutation'] = df['date_mutation'].dt.day
    # colonne saison (catégorielle)
    df['saison'] = df['mois_mutation'].apply(get_season)

    # si tu veux aussi des indicateurs binaires :
    df['isWinter']   = (df['saison'] == 'hiver').astype(int)
    df['isSpring']   = (df['saison'] == 'printemps').astype(int)
    df['isSummer']   = (df['saison'] == 'ete').astype(int)
    df1['isAutumn']   = (df['saison'] == 'automne').astype(int)

else:
    print("'date_mutation' n'existe pas dans le DataFrame.")

# df.drop(columns=['date_mutation'], inplace=True)
df.head()

In [ ]:
# Colonnes numériques de ton dataframe
num_cols = [
    'valeur_fonciere',
    'code_type_local',
    'surface_reelle_bati',
    'nombre_pieces_principales',
    'surface_terrain'
]

# Fonction suppression outliers avec IQR
def remove_outliers_iqr(df, columns):
    df_clean = df.copy()
    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df_clean = df_clean[(df_clean[col] >= lower) & (df_clean[col] <= upper)]
    return df_clean

# Application sur ton dataset
df_no_outliers = remove_outliers_iqr(df, num_cols)
df=df_no_outliers

print("Shape avant :", df.shape)
print("Shape après :", df_no_outliers.shape)



In [ ]:
# on supprime les valeurs manquantes de l'attribut a prédire
df = df.dropna(subset=['valeur_fonciere'])

from sklearn.preprocessing import StandardScaler, LabelEncoder
num_cols = [
    'surface_reelle_bati',
    'nombre_pieces_principales',
    'surface_terrain'
]

cat_cols = [
    'code_postal',
    'commune',
    'code_departement',
    'code_commune',
    'code_type_local'
]

# numériques → médiane
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

# catégorielles → UNKNOWN
for col in cat_cols:
    df[col] = df[col].fillna("UNKNOWN")


label_encoders = {}
# Encodage des colonnes catégorielles
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# Normalisation des colonnes numériques
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

df_ready = df[num_cols + cat_cols + ["valeur_fonciere"]]
df_ready.head(200)

In [ ]:
df_ready.to_csv('learning_data_house.csv', index=False)